In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, precision_score, f1_score, recall_score

In [2]:
data = pd.read_csv('FastagFraudDetection.csv')

In [27]:
data=pd.DataFrame(data)
data

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van,Output
0,1,0.0,0.0,350,120,65,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,1.0,1.0,120,100,78,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,3,1.0,1.0,0,0,53,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
3,4,1.0,0.0,350,120,92,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,5,0.0,2.0,140,100,60,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,1.0,0.0,330,330,81,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4996,4997,0.0,2.0,125,125,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4997,4998,1.0,2.0,115,115,93,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4998,4999,0.0,0.0,145,145,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [9]:
data.shape

(5000, 13)

In [5]:
data['Vehicle_Type'].unique()

array(['Bus ', 'Car', 'Motorcycle', 'Truck', 'Van', 'Sedan', 'SUV'],
      dtype=object)

In [28]:
Lane_order=['Express', 'Regular']
Vehicle_Dimensions_order=['Large', 'Small', 'Medium']
Fraud_indicator_order=['Not Fraud','Fraud']

In [7]:
ohe = OneHotEncoder()
encode0 = ohe.fit_transform(data[['Vehicle_Type']]).toarray()

In [8]:
feature_labels = ohe.categories_
np.array(feature_labels).ravel()

array(['Bus ', 'Car', 'Motorcycle', 'SUV', 'Sedan', 'Truck', 'Van'],
      dtype=object)

In [10]:
feature_labels = np.array(feature_labels).ravel()
print(feature_labels)

['Bus ' 'Car' 'Motorcycle' 'SUV' 'Sedan' 'Truck' 'Van']


In [11]:
features = pd.DataFrame(encode0, columns = feature_labels)

In [12]:
df_new = pd.concat([data, features], axis=1)

In [13]:
new_dataset=df_new.drop(['Timestamp','FastagID','Vehicle_Type','TollBoothID','Geographical_Location','Vehicle_Plate_Number'], axis=1)

In [14]:
new_dataset


,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
0,1,Express,Large,350,120,65,Fraud,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Regular,Small,120,100,78,Fraud,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,Regular,Small,0,0,53,Not Fraud,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Regular,Large,350,120,92,Fraud,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,Express,Medium,140,100,60,Fraud,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Regular,Large,330,330,81,Not Fraud,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4996,4997,Express,Medium,125,125,64,Not Fraud,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4997,4998,Regular,Medium,115,115,93,Not Fraud,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4998,4999,Express,Large,145,145,57,Not Fraud,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
encode1 = OrdinalEncoder(categories=[Lane_order])
encode2 = OrdinalEncoder(categories=[Vehicle_Dimensions_order])
encode3 = OrdinalEncoder(categories=[Fraud_indicator_order])


In [16]:
encode1.fit(new_dataset[['Lane_Type']])
encode2.fit(new_dataset[['Vehicle_Dimensions']])
encode3.fit(new_dataset[['Fraud_indicator']])

OrdinalEncoder(categories=[['Not Fraud', 'Fraud']])

In [17]:
new_lane=pd.DataFrame(encode1.transform(new_dataset[['Lane_Type']]))
new_dimensuions=pd.DataFrame(encode2.transform(new_dataset[['Vehicle_Dimensions']]))
new_fraud_indicator=pd.DataFrame(encode3.transform(new_dataset[['Fraud_indicator']]))

In [18]:
new_dataset['Lane_Type']= new_lane
new_dataset['Vehicle_Dimensions']= new_dimensuions
new_dataset['Fraud_indicator']=new_fraud_indicator

In [19]:
new_dataset

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
0,1,0.0,0.0,350,120,65,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,1.0,120,100,78,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,1.0,0,0,53,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1.0,0.0,350,120,92,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,2.0,140,100,60,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,1.0,0.0,330,330,81,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4996,4997,0.0,2.0,125,125,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4997,4998,1.0,2.0,115,115,93,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4998,4999,0.0,0.0,145,145,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
# handlling missing values in dataset with dropping method
data = new_dataset.dropna(how='any')

In [21]:
# checking the missing values of each column
data.isnull().sum()

Transaction_ID        0
Lane_Type             0
Vehicle_Dimensions    0
Transaction_Amount    0
Amount_paid           0
Vehicle_Speed         0
Fraud_indicator       0
Bus                   0
Car                   0
Motorcycle            0
SUV                   0
Sedan                 0
Truck                 0
Van                   0
dtype: int64

In [24]:
import numpy as np

# Assuming `data` is your DataFrame
data['Output'] = np.where(data['Fraud_indicator'] == 'Fraud', 1, 0)


In [25]:
# checking the missing values of each column
data.isnull().sum()

Transaction_ID        0
Lane_Type             0
Vehicle_Dimensions    0
Transaction_Amount    0
Amount_paid           0
Vehicle_Speed         0
Fraud_indicator       0
Bus                   0
Car                   0
Motorcycle            0
SUV                   0
Sedan                 0
Truck                 0
Van                   0
Output                0
dtype: int64

In [29]:
data['Fraud_indicator'].value_counts()

Fraud_indicator
0.0    4017
1.0     983
Name: count, dtype: int64

In [32]:
# separating data for analysis
legit = data[data.Fraud_indicator == 0]
fraud = data[data.Fraud_indicator == 1]

In [33]:
print(legit.shape)
print(fraud.shape)


(4017, 15)
(983, 15)


In [34]:
#statistical method of the data
legit.Transaction_Amount.describe()

count    4017.000000
mean      153.110530
std       114.435986
min         0.000000
25%        90.000000
50%       125.000000
75%       290.000000
max       350.000000
Name: Transaction_Amount, dtype: float64

In [35]:
fraud.Transaction_Amount.describe()

count    983.000000
mean     193.555443
std       97.465586
min       60.000000
25%      120.000000
50%      145.000000
75%      300.000000
max      350.000000
Name: Transaction_Amount, dtype: float64

In [36]:
#compare the values for both transactions
data.groupby('Fraud_indicator').mean()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van,Output
Fraud_indicator,,,,,,,,,,,,,,
0.0,2618.246950,0.588748,0.865820,153.110530,153.11053,67.731392,0.134180,0.147374,0.177745,0.131939,0.137665,0.138412,0.132686,0.0
1.0,2019.330621,0.501526,0.819939,193.555443,92.83825,68.340793,0.180061,0.124110,0.000000,0.187182,0.163784,0.160732,0.184130,0.0


In [37]:
legit_sample = legit.sample(n=983)

In [38]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [39]:
new_dataset.head()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van,Output
4907,4908,1.0,0.0,145,145,72,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
335,336,1.0,0.0,180,180,81,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4493,4494,1.0,2.0,110,110,76,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4777,4778,1.0,1.0,0,0,79,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
2413,2414,1.0,2.0,115,115,75,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [41]:
#dristibution of legit and fraud transactions in new dataset
new_dataset['Fraud_indicator'].value_counts()

Fraud_indicator
0.0    983
1.0    983
Name: count, dtype: int64

In [43]:
new_dataset.groupby('Fraud_indicator').mean()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van,Output
Fraud_indicator,,,,,,,,,,,,,,
0.0,2591.727365,0.608342,0.887080,145.996948,145.996948,68.712106,0.120041,0.148525,0.191251,0.145473,0.144456,0.121058,0.129196,0.0
1.0,2019.330621,0.501526,0.819939,193.555443,92.838250,68.340793,0.180061,0.124110,0.000000,0.187182,0.163784,0.160732,0.184130,0.0


In [44]:
X = new_dataset.drop(columns='Fraud_indicator', axis=1)
Y = new_dataset['Fraud_indicator']

In [45]:
print(X)

      Transaction_ID  Lane_Type  Vehicle_Dimensions  Transaction_Amount  \
4907            4908        1.0                 0.0                 145   
335              336        1.0                 0.0                 180   
4493            4494        1.0                 2.0                 110   
4777            4778        1.0                 1.0                   0   
2413            2414        1.0                 2.0                 115   
...              ...        ...                 ...                 ...   
4957            4958        1.0                 0.0                 330   
4962            4963        0.0                 2.0                 115   
4970            4971        0.0                 0.0                 145   
4975            4976        1.0                 2.0                 125   
4999            5000        1.0                 0.0                 330   

      Amount_paid  Vehicle_Speed  Bus   Car  Motorcycle  SUV  Sedan  Truck  \
4907          145    

In [46]:
print(Y)

4907    0.0
335     0.0
4493    0.0
4777    0.0
2413    0.0
       ... 
4957    1.0
4962    1.0
4970    1.0
4975    1.0
4999    1.0
Name: Fraud_indicator, Length: 1966, dtype: float64


In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8, stratify=Y, random_state=2)

In [48]:
print(X.shape, X_train.shape, X_test.shape)

(1966, 14) (1572, 14) (394, 14)


In [49]:
pd.DataFrame(X_train)


,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van,Output
2401,2402,1.0,0.0,330,330,92,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3758,3759,0.0,2.0,120,100,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2181,2182,1.0,2.0,125,90,95,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4626,4627,0.0,2.0,115,115,67,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2990,2991,0.0,0.0,350,350,74,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,1274,1.0,0.0,150,150,96,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4026,4027,0.0,0.0,300,300,82,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3778,3779,0.0,2.0,130,130,50,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1989,1990,1.0,1.0,70,70,84,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [50]:
pd.DataFrame(Y_train)

,Fraud_indicator
2401,0.0
3758,1.0
2181,1.0
4626,0.0
2990,0.0
...,...
1273,0.0
4026,0.0
3778,0.0
1989,0.0


In [51]:
Models = {
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Logistic Regression":LogisticRegression(),
    "SVM Classification": SVC()
}

for i in range (len(list(Models))):
    Model=list(Models.values())[i]

    #train Model
    Model.fit(X_train, Y_train)

    #Make predictions
    Y_train_pred = Model.predict(X_train)
    Y_test_pred = Model.predict(X_test)

    #Training Performance
    model_train_Accuracy = accuracy_score(Y_train, Y_train_pred)
    model_train_Precision = precision_score(Y_train, Y_train_pred)
    model_train_recall = recall_score(Y_train, Y_train_pred)
    model_train_F1 = f1_score(Y_train, Y_train_pred, average='weighted')

    #Testing Performance
    model_test_Accuracy = accuracy_score(Y_test, Y_test_pred)
    model_test_Precision = precision_score(Y_test, Y_test_pred)
    model_test_recall = recall_score(Y_test, Y_test_pred)
    model_test_F1 = f1_score(Y_test, Y_test_pred, average='weighted')

    print(list(Models.keys())[i])

    print("Models Performance for Training Set")
    print("- Accuracy: {:.4f}".format(model_train_Accuracy))
    print("- Precision: {:.4f}".format(model_train_Precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    print("- F1 Score: {:.4f}".format(model_train_F1))


    print("--------------------------")

    print("Models Performance for Testing Set")
    print("- Accuracy: {:.4f}".format(model_test_Accuracy))
    print("- Precision: {:.4f}".format(model_test_Precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    print("- F1 Score: {:.4f}".format(model_test_F1))

    print('='*35)
    print('\n')

Decision Tree
Models Performance for Training Set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9797
- Precision: 0.9797
- Recall: 0.9797
- F1 Score: 0.9797


Random Forest
Models Performance for Training Set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9746
- Precision: 0.9845
- Recall: 0.9645
- F1 Score: 0.9746


Logistic Regression
Models Performance for Training Set
- Accuracy: 0.9758
- Precision: 0.9987
- Recall: 0.9529
- F1 Score: 0.9758
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9848
- Precision: 1.0000
- Recall: 0.9695
- F1 Score: 0.9848




C:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM Classification
Models Performance for Training Set
- Accuracy: 0.6940
- Precision: 0.7424
- Recall: 0.5941
- F1 Score: 0.6909
--------------------------
Models Performance for Testing Set
- Accuracy: 0.7107
- Precision: 0.7578
- Recall: 0.6193
- F1 Score: 0.7082


